# RSA Blind Signatures

In [24]:
import random 
from Crypto.Util.number import GCD, inverse
from Crypto.Hash import SHA256



# Generate a pair of RSA keys
def generate_key_pair(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537 # public exponent
    d = inverse(e, phi) # private exponent inverse of u modulo v exists iff GCD(u, v) = 1
    return (e, n), (d, n)

# Sign a message using RSA
# def sign_message(message, private_key):
#     d, n = private_key
#     signature = pow(message, d, n)
#     return signature

def sign_message(message, private_key):
    d, n = private_key
    signature = pow(message, d, n)
    return signature


# def sign_message(message, private_key):
#     d, n = private_key
#     hashed_message = int.from_bytes(SHA256.new(message.to_bytes(8, 'big')).digest(), 'big')
#     signature = pow(hashed_message, d, n)
#     return signature


# Blind a message using RSA
# def blind_message(message, public_key):
#     e, n = public_key
#     r = random.randint(1, n-1)
#     while GCD(r, n) != 1:
#         r = random.randint(1, n-1)
#     blinded_message = (message * pow(r, e, n)) % n
#     return blinded_message, r

def blind_message(message, public_key):
    e, n = public_key
    r = random.randint(1, n-1)
    while GCD(r, n) != 1:
        r = random.randint(1, n-1)
    blinded_message = (pow(message, e, n) * pow(r, e, n)) % n
    return blinded_message, r


# Unblind a signature using RSA
# def unblind_signature(blind_signature, blinding_factor, public_key):
#     e, n = public_key
#     unblinded_signature = (blind_signature * inverse(blinding_factor, n)) % n
#     return unblinded_signature

def unblind_signature(blind_signature, blinding_factor, public_key):
    e, n = public_key
    unblinded_signature = (blind_signature * inverse(pow(blinding_factor, e, n), n)) % n
    return unblinded_signature

# Verify a signature using RSA

# def verify_signature(message, signature, public_key):
#     e, n = public_key
#     hashed_message = int.from_bytes(SHA256.new(message.to_bytes(8, 'big')).digest(), 'big')
#     if pow(signature, e, n) == hashed_message:
#         return True
#     else:
#         return False

def verify_signature(message, signature, public_key):
    e, n = public_key
    if pow(signature, e, n) % n == message % n:
        return True
    else:
        return False
    
# def verify_signature(message, signature, public_key):
#     e, n = public_key
#     if pow(signature, e, n) == pow(message, e, n):
#         return True
#     else:
#         return False


# def verify_signature(message, signature, public_key):
#     e, n = public_key
#     if pow(signature, e, n) == message:
#         return True
#     else:
#         return False


# Example
# =======
# =======
# =======
# Generate a pair of RSA keys
public_key, private_key = generate_key_pair(17, 23)
print("Public key: ", public_key)
print("Private key: ", private_key)


# Choose a message to sign
message = 123456789
print("Message: ", message)

# Blind the message using the public key
print("Blinding the message",message, "with public key", public_key)
blinded_message, blinding_factor = blind_message(message, public_key)
print("Blinded message: ", blinded_message)
print("Blinding factor: ", blinding_factor)

# Send the blinded message to the signer and receive a blind signature
print("signing the blind message", blinded_message, "with private key", private_key)
blind_signature = sign_message(blinded_message, private_key)
print("Blind signature: ", blind_signature)

# Unblind the blind signature using the blinding factor and public key
unblinded_signature = unblind_signature(blind_signature, blinding_factor, public_key)
print("Unblinded signature: ", unblinded_signature)

# Verify the signature using the public key
if verify_signature(message, unblinded_signature, public_key):
    print("Signature is valid!")
else:
    print("Signature is not valid.")


Public key:  (65537, 391)
Private key:  (65, 391)
Message:  123456789
Blinding the message 123456789 with public key (65537, 391)
Blinded message:  205
Blinding factor:  1
signing the blind message 205 with private key (65, 391)
Blind signature:  103
Unblinded signature:  103
Signature is not valid.


# ElGamal Blind Signatures

In [19]:
import random
from Crypto.Util.number import GCD, inverse


# Generate an ElGamal key pair
def generate_key_pair(p, g, x):
    y = pow(g, x, p) # public key
    return (p, g, y), x

# Sign a message using ElGamal
def sign_message(message, private_key):
    p, g, x = private_key
    k = random.randint(1, p-2)
    r = pow(g, k, p)
    s = (message - x * r) * inverse(k, p-1) % (p-1)
    return r, s

# Blind a message using ElGamal
def blind_message(message, public_key):
    p, g, y = public_key
    k = random.randint(1, p-2)
    r = pow(g, k, p)
    c = (message * pow(y, k, p)) % p
    return c, r

# Unblind a signature using ElGamal
def unblind_signature(blind_signature, blinding_factor, public_key):
    p, g, y = public_key
    c, r = blind_signature
    unblinded_signature = ((c % p) * inverse(pow(blinding_factor, p-1, p), p)) % p
    return unblinded_signature, r

# Verify a signature using ElGamal
def verify_signature(message, signature, public_key):
    p, g, y = public_key
    r, s = signature
    left = pow(g, message, p)
    right = (pow(y, r, p) * pow(r, s, p)) % p
    if left == right:
        return True
    else:
        return False


# Generate an ElGamal key pair
public_key, private_key = generate_key_pair(17, 3, 5)
# ElGamal key pair p = 47, g = 5, x = 11 (p,g,x)
# p = prime number
# g = generator
# x = private key

# Choose a message to sign
message = 27

# Blind the message using the public key
blinded_message, blinding_factor = blind_message(message, public_key)

# Send the blinded message to the signer and receive a blind signature
blind_signature = sign_message(blinded_message, private_key)

# Unblind the blind signature using the blinding factor and public key
unblinded_signature = unblind_signature(blind_signature, blinding_factor, public_key)

# Verify the signature using the public key
if verify_signature(message, unblinded_signature, public_key):
    print("Signature is valid!")
else:
    print("Signature is not valid.")


TypeError: cannot unpack non-iterable int object

RSA Blind Signatures with Hash